## Prepare the workspace

In [2]:
# Check torch version and CUDA status if GPU is enabled.
import torch
print(torch.__version__)
print(torch.cuda.is_available()) # Should return True when GPU is enabled. 

2.0.1
11.7


# Developing an AI application

Going forward, AI algorithms will be incorporated into more and more
everyday applications. For example, you might want to include an image
classifier in a smart phone app. To do this, you'd use a deep learning
model trained on hundreds of thousands of images as part of the overall
application architecture. A large part of software development in the
future will be using these types of models as common parts of
applications.

In this project, you'll train an image classifier to recognize different
species of flowers. You can imagine using something like this in a phone
app that tells you the name of the flower your camera is looking at. In
practice you'd train this classifier, then export it for use in your
application. We'll be using [this
dataset](http://www.robots.ox.ac.uk/~vgg/data/flowers/102/index.html) of
102 flower categories, you can see a few examples below.

<img src='assets/Flowers.png' width=500px>

The project is broken down into multiple steps:

-   Load and preprocess the image dataset
-   Train the image classifier on your dataset
-   Use the trained classifier to predict image content

We'll lead you through each part which you'll implement in Python.

When you've completed this project, you'll have an application that can
be trained on any set of labeled images. Here your network will be
learning about flowers and end up as a command line application. But,
what you do with your new skills depends on your imagination and effort
in building a dataset. For example, imagine an app where you take a
picture of a car, it tells you what the make and model is, then looks up
information about it. Go build your own dataset and make something new.

First up is importing the packages you'll need. It's good practice to
keep all the imports at the beginning of your code. As you work through
this notebook and find you need to import a package, make sure to add
the import up here.

In [ ]:
# Imports here
import torch
import torch.nn.functional as F
from torch import nn, optim
from torchvision import datasets, transforms, models
import json
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt

## Load the data

Here you'll use `torchvision` to load the data
([documentation](http://pytorch.org/docs/0.3.0/torchvision/index.html)).
The data should be included alongside this notebook, otherwise you can
[download it
here](https://s3.amazonaws.com/content.udacity-data.com/nd089/flower_data.tar.gz).

If you do not find the `flowers/` dataset in the current directory,
**/workspace/home/aipnd-project/**, you can download it using the
following commands.

**Method 1**

``` bash
!wget 'https://s3.amazonaws.com/content.udacity-data.com/nd089/flower_data.tar.gz'
!unlink flowers
!mkdir flowers && tar -xzf flower_data.tar.gz -C flowers
```

**Method 2**

``` bash
!cp -r /data/ .
```

## Data Description

The dataset is split into three parts, training, validation, and
testing. For the training, you'll want to apply transformations such as
random scaling, cropping, and flipping. This will help the network
generalize leading to better performance. You'll also need to make sure
the input data is resized to 224x224 pixels as required by the
pre-trained networks.

The validation and testing sets are used to measure the model's
performance on data it hasn't seen yet. For this you don't want any
scaling or rotation transformations, but you'll need to resize then crop
the images to the appropriate size.

The pre-trained networks you'll use were trained on the ImageNet dataset
where each color channel was normalized separately. For all three sets
you'll need to normalize the means and standard deviations of the images
to what the network expects. For the means, it's `[0.485, 0.456, 0.406]`
and for the standard deviations `[0.229, 0.224, 0.225]`, calculated from
the ImageNet images. These values will shift each color channel to be
centered at 0 and range from -1 to 1.

In [ ]:
data_dir = 'flower_data'
train_dir = data_dir + '/train'
valid_dir = data_dir + '/valid'
test_dir = data_dir + '/test'

In [ ]:
# TODO: Define your transforms for the training, validation, and testing sets

data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ]),
    'valid': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
}

# TODO: Load the datasets with ImageFolder

image_datasets = {
    'train': datasets.ImageFolder(train_dir, transform=data_transforms['train']),
    'valid': datasets.ImageFolder(valid_dir, transform=data_transforms['valid']),
    'test': datasets.ImageFolder(test_dir, transform=data_transforms['test'])
}

# TODO: Using the image datasets and the trainforms, define the dataloaders

dataloaders = {
    'train': torch.utils.data.DataLoader(image_datasets['train'], batch_size=32, shuffle=True),
    'valid': torch.utils.data.DataLoader(image_datasets['valid'], batch_size=32, shuffle=True),
    'test': torch.utils.data.DataLoader(image_datasets['test'], batch_size=32, shuffle=True)
}


### Label mapping

You'll also need to load in a mapping from category label to category
name. You can find this in the file `cat_to_name.json`. It's a JSON
object which you can read in with the [`json`
module](https://docs.python.org/2/library/json.html). This will give you
a dictionary mapping the integer encoded categories to the actual names
of the flowers.

In [ ]:
import json

with open('cat_to_name.json', 'r') as f:
    cat_to_name = json.load(f)

# Building and training the classifier

Now that the data is ready, it's time to build and train the classifier.
As usual, you should use one of the pretrained models from
`torchvision.models` to get the image features. Build and train a new
feed-forward classifier using those features.

We're going to leave this part up to you. Refer to [the
rubric](https://review.udacity.com/#!/rubrics/1663/view) for guidance on
successfully completing this section. Things you'll need to do:

-   Load a [pre-trained
    network](http://pytorch.org/docs/master/torchvision/models.html) (If
    you need a starting point, the VGG networks work great and are
    straightforward to use)
-   Define a new, untrained feed-forward network as a classifier, using
    ReLU activations and dropout
-   Train the classifier layers using backpropagation using the
    pre-trained network to get the features
-   Track the loss and accuracy on the validation set to determine the
    best hyperparameters

We've left a cell open for you below, but use as many as you need. Our
advice is to break the problem up into smaller parts you can run
separately. Check that each part is doing what you expect, then move on
to the next. You'll likely find that as you work through each part,
you'll need to go back and modify your previous code. This is totally
normal!

When training make sure you're updating only the weights of the
feed-forward network. You should be able to get the validation accuracy
above 70% if you build everything right. Make sure to try different
hyperparameters (learning rate, units in the classifier, epochs, etc) to
find the best model. Save those hyperparameters to use as default values
in the next part of the project.

One last important tip if you're using the workspace to run your code:
To avoid having your workspace disconnect during the long-running tasks
in this notebook, please read in the earlier page in this lesson called
Intro to GPU Workspaces about Keeping Your Session Active.

## Note for Workspace users:

If your network is over 1 GB when saved as a checkpoint, there might be
issues with saving backups in your workspace. Typically this happens
with wide dense layers after the convolutional layers. If your saved
checkpoint is larger than 1 GB (you can open a terminal and check with
`ls -lh`), you should reduce the size of your hidden layers and train
again.

In [ ]:
# Build and Train the Network

def train_network(model, dataloaders, criterion, optimizer, epochs=5):
    # Move model to GPU if available
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    # Loop through epochs
    for epoch in range(epochs):
        model.train()  # Set the model to training mode
        running_loss = 0.0
        correct = 0
        total = 0

        for inputs, labels in dataloaders['train']:
            # Move data to GPU if available
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()  # Zero the gradients

            # Forward pass
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            # Backward pass
            loss.backward()
            optimizer.step()

            running_loss += loss.item()  # Accumulate the loss
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        # Calculate training accuracy
        train_accuracy = correct / total
        print(f"Epoch {epoch + 1}/{epochs}")
        print(f"Training Loss: {running_loss / len(dataloaders['train']):.4f}, Accuracy: {train_accuracy:.4f}")
        
    return model  # Return the trained model



## Testing your network

It's good practice to test your trained network on test data, images the
network has never seen either in training or validation. This will give
you a good estimate for the model's performance on completely new
images. Run the test images through the network and measure the
accuracy, the same way you did validation. You should be able to reach
around 70% accuracy on the test set if the model has been trained well.

In [ ]:
# TODO: Do validation on the test set
# Validate the Network on the Test Set
def test_network(model, dataloaders, criterion):
    model.eval()  # Set the model to evaluation mode
    running_loss = 0.0
    correct = 0
    total = 0

    # No gradient calculation during test
    with torch.no_grad():
        for inputs, labels in dataloaders['test']:
            inputs, labels = inputs.to(device), labels.to(device)

            # Forward pass
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            running_loss += loss.item()

            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    # Calculate test accuracy
    test_accuracy = correct / total
    print(f"Test Loss: {running_loss / len(dataloaders['test']):.4f}, Accuracy: {test_accuracy:.4f}")
    return test_accuracy  # Return test accuracy


## Save the checkpoint

Now that your network is trained, save the model so you can load it
later for making predictions. You probably want to save other things
such as the mapping of classes to indices which you get from one of the
image datasets: `image_datasets['train'].class_to_idx`. You can attach
this to the model as an attribute which makes inference easier later on.

`model.class_to_idx = image_datasets['train'].class_to_idx`

Remember that you'll want to completely rebuild the model later so you
can use it for inference. Make sure to include any information you need
in the checkpoint. If you want to load the model and keep training,
you'll want to save the number of epochs as well as the optimizer state,
`optimizer.state_dict`. You'll likely want to use this trained model in
the next part of the project, so best to save it now.

In [ ]:
# TODO: Save the checkpoint
# Load the checkpoint to resume training or use the model for inference
# Save the model checkpoint after training
import torch
import torch.nn.functional as F

def save_checkpoint(model, optimizer, epoch, loss, file_path='model_checkpoint.pth'):
    """
    Save the model checkpoint, including the model's state_dict, optimizer's state_dict, 
    and other relevant information like epoch and loss.
    """
    checkpoint = {
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'loss': loss
    }
    torch.save(checkpoint, file_path)
    print(f"Checkpoint saved to {file_path}")

def load_checkpoint(model, optimizer, file_path='model_checkpoint.pth'):
    """
    Load the model checkpoint to resume training or for inference.
    """
    checkpoint = torch.load(file_path)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    epoch = checkpoint['epoch']
    loss = checkpoint['loss']
    print(f"Checkpoint loaded from {file_path}. Starting from epoch {epoch} with loss {loss}")
    return model, optimizer, epoch, loss

def handle_shape_mismatch(tensor_a, tensor_b):
    """
    Handle tensor shape mismatches by padding the smaller tensor to match the size of the larger one.
    """
    if tensor_a.size(0) != tensor_b.size(0):
        if tensor_a.size(0) > tensor_b.size(0):
            tensor_b = F.pad(tensor_b, (0, 0, 0, tensor_a.size(0) - tensor_b.size(0)))
            print(f"Padded tensor_b to match tensor_a: {tensor_b.shape}")
        elif tensor_b.size(0) > tensor_a.size(0):
            tensor_a = F.pad(tensor_a, (0, 0, 0, tensor_b.size(0) - tensor_a.size(0)))
            print(f"Padded tensor_a to match tensor_b: {tensor_a.shape}")
    return tensor_a, tensor_b

# Example tensors
tensor_a = torch.randn(4, 5)
tensor_b = torch.randn(3, 5)

# Handle shape mismatch
tensor_a, tensor_b = handle_shape_mismatch(tensor_a, tensor_b)

# Now you can safely perform operations
result = tensor_a + tensor_b
print("Result of operation:", result)


## Loading the checkpoint

At this point it's good to write a function that can load a checkpoint
and rebuild the model. That way you can come back to this project and
keep working on it without having to retrain the network.

In [ ]:
# TODO: Write a function that loads a checkpoint and rebuilds the model
import torch

# Function to rebuild the model and load the checkpoint
def load_model_from_checkpoint(model_class, model_params, checkpoint_path='model_checkpoint.pth'):
    """
    Loads a model from a checkpoint and rebuilds it.
    
    Args:
        model_class (torch.nn.Module): The model class used to rebuild the model.
        model_params (dict): Parameters required to initialize the model class.
        checkpoint_path (str): The path to the checkpoint file.
        
    Returns:
        model: The model restored to the state saved in the checkpoint.
        optimizer: The optimizer with the state loaded.
        epoch: The epoch from the checkpoint.
        loss: The loss from the checkpoint.
    """
    # Rebuild the model
    model = model_class(**model_params)  # Initialize the model with required parameters
    
    # Load the checkpoint
    checkpoint = torch.load(checkpoint_path)
    
    # Load model weights (state_dict) from checkpoint
    model.load_state_dict(checkpoint['model_state_dict'])
    
    # Initialize the optimizer (this should match the optimizer used during training)
    optimizer = model_params['optimizer_class'](model.parameters(), **model_params['optimizer_params'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    
    # Get epoch and loss from the checkpoint
    epoch = checkpoint['epoch']
    loss = checkpoint['loss']
    
    print(f"Checkpoint loaded from {checkpoint_path}. Model trained until epoch {epoch} with loss {loss:.4f}")
    
    return model, optimizer, epoch, loss


# Inference for classification

Now you'll write a function to use a trained network for inference. That
is, you'll pass an image into the network and predict the class of the
flower in the image. Write a function called `predict` that takes an
image and a model, then returns the top $K$ most likely classes along
with the probabilities. It should look like

``` python
probs, classes = predict(image_path, model)
print(probs)
print(classes)
> [ 0.01558163  0.01541934  0.01452626  0.01443549  0.01407339]
> ['70', '3', '45', '62', '55']
```

First you'll need to handle processing the input image such that it can
be used in your network.

## Image Preprocessing

You'll want to use `PIL` to load the image
([documentation](https://pillow.readthedocs.io/en/latest/reference/Image.html)).
It's best to write a function that preprocesses the image so it can be
used as input for the model. This function should process the images in
the same manner used for training.

First, resize the images where the shortest side is 256 pixels, keeping
the aspect ratio. This can be done with the
[`thumbnail`](http://pillow.readthedocs.io/en/3.1.x/reference/Image.html#PIL.Image.Image.thumbnail)
or
[`resize`](http://pillow.readthedocs.io/en/3.1.x/reference/Image.html#PIL.Image.Image.thumbnail)
methods. Then you'll need to crop out the center 224x224 portion of the
image.

Color channels of images are typically encoded as integers 0-255, but
the model expected floats 0-1. You'll need to convert the values. It's
easiest with a Numpy array, which you can get from a PIL image like so
`np_image = np.array(pil_image)`.

As before, the network expects the images to be normalized in a specific
way. For the means, it's `[0.485, 0.456, 0.406]` and for the standard
deviations `[0.229, 0.224, 0.225]`. You'll want to subtract the means
from each color channel, then divide by the standard deviation.

And finally, PyTorch expects the color channel to be the first dimension
but it's the third dimension in the PIL image and Numpy array. You can
reorder dimensions using
[`ndarray.transpose`](https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.ndarray.transpose.html).
The color channel needs to be first and retain the order of the other
two dimensions.

In [ ]:
import torch
from torchvision import transforms
from PIL import Image

# Function to process an image
def process_image(image_path, input_size=(224, 224), mean=None, std=None):
    """
    Process a PIL image to be used as input for a PyTorch model.
    
    Args:
        image_path (str): Path to the image file.
        input_size (tuple): Desired image size (width, height) to resize to.
        mean (tuple): Mean values for normalization (e.g., for pretrained models).
        std (tuple): Standard deviation values for normalization (e.g., for pretrained models).
    
    Returns:
        torch.Tensor: The processed image as a tensor ready for model input.
    """
    # Load the image
    image = Image.open(image_path)
    
    # Convert the image to RGB (in case it's RGBA or has an alpha channel)
    image = image.convert('RGB')
    
    # Define the transformation pipeline
    transform_list = []
    
    # Resize the image to the desired input size
    transform_list.append(transforms.Resize(input_size))
    
    # Convert the image to tensor (scaling pixel values to [0, 1])
    transform_list.append(transforms.ToTensor())
    
    # Normalize the image if mean and std are provided (commonly used for pretrained models)
    if mean is not None and std is not None:
        transform_list.append(transforms.Normalize(mean=mean, std=std))
    
    # Create the transformation pipeline
    transform = transforms.Compose(transform_list)
    
    # Apply the transformations to the image
    image_tensor = transform(image)
    
    # Add batch dimension (models expect a batch of images, so we need to add a batch dimension)
    image_tensor = image_tensor.unsqueeze(0)  # Shape will be [1, 3, 224, 224]
    
    return image_tensor

# Example usage:
image_path = 'assets/Flowers.png'

# Example mean and std for ImageNet models (if you're using a pretrained model like ResNet)
mean = [0.485, 0.456, 0.406]  # for RGB channels
std = [0.229, 0.224, 0.225]   # for RGB channels

# Process the image
processed_image = process_image(image_path, input_size=(224, 224), mean=mean, std=std)

# Now `processed_image` is ready to be passed into a model
print(processed_image.shape)  # Should print torch.Size([1, 3, 224, 224])





To check your work, the function below converts a PyTorch tensor and
displays it in the notebook. If your `process_image` function works,
running the output through this function should return the original
image (except for the cropped out portions).

In [ ]:
def imshow(image, ax=None, title=None):
    """Imshow for Tensor."""
    if ax is None:
        fig, ax = plt.subplots()
    
    # PyTorch tensors assume the color channel is the first dimension
    # but matplotlib assumes is the third dimension
    image = image.numpy().transpose((1, 2, 0))
    
    # Undo preprocessing
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    image = std * image + mean
    
    # Image needs to be clipped between 0 and 1 or it looks like noise when displayed
    image = np.clip(image, 0, 1)
    
    ax.imshow(image)
    
    return ax

## Class Prediction

Once you can get images in the correct format, it's time to write a
function for making predictions with your model. A common practice is to
predict the top 5 or so (usually called top-$K$) most probable classes.
You'll want to calculate the class probabilities then find the $K$
largest values.

To get the top $K$ largest values in a tensor use
[`x.topk(k)`](http://pytorch.org/docs/master/torch.html#torch.topk).
This method returns both the highest `k` probabilities and the indices
of those probabilities corresponding to the classes. You need to convert
from these indices to the actual class labels using `class_to_idx` which
hopefully you added to the model or from an `ImageFolder` you used to
load the data ([see here](#Save-the-checkpoint)). Make sure to invert
the dictionary so you get a mapping from index to class as well.

Again, this method should take a path to an image and a model
checkpoint, then return the probabilities and classes.

``` python
probs, classes = predict(image_path, model)
print(probs)
print(classes)
> [ 0.01558163  0.01541934  0.01452626  0.01443549  0.01407339]
> ['70', '3', '45', '62', '55']
```

In [ ]:
import torch
from torchvision import models, transforms
from PIL import Image
import json
import requests

# Function to process an image (resize, tensor conversion, normalization)

def process_image(image_path, input_size=(224, 224), mean=None, std=None):
    image = Image.open(image_path)
    
    # Convert image to RGB if it's in RGBA format (4 channels)
    if image.mode != 'RGB':
        image = image.convert('RGB')
    
    transform_list = [
        transforms.Resize(input_size),
        transforms.ToTensor(),
    ]
    
    if mean is not None and std is not None:
        transform_list.append(transforms.Normalize(mean=mean, std=std))
    
    transform = transforms.Compose(transform_list)
    image_tensor = transform(image)
    
    return image_tensor


# Function to load and predict class of an image
def predict_class(image_path, model=None, input_size=(224, 224), mean=None, std=None):
    # Step 1: Preprocess the image
    processed_image = process_image(image_path, input_size, mean, std)
    
    # Add batch dimension (models expect a batch of images)
    processed_image = processed_image.unsqueeze(0)  # Shape [1, 3, 224, 224]
    
    # Step 2: Load the model (if not passed)
    if model is None:
        model = models.resnet18(pretrained=True)
        model.eval()  # Set to evaluation mode
    
    # Step 3: Predict the class
    with torch.no_grad():  # No gradients needed for inference
        output = model(processed_image)
    
    # Step 4: Get the predicted class (the index of the max logit)
    _, predicted_class = torch.max(output, 1)
    
    # Get the label (if using ImageNet labels)
    # Download ImageNet class labels (if needed)
    if not hasattr(predict_class, 'imagenet_class_index'):
        url = 'https://storage.googleapis.com/download.tensorflow.org/data/imagenet_class_index.json'
        response = requests.get(url)
        predict_class.imagenet_class_index = response.json()
    
    class_idx = predicted_class.item()
    class_label = predict_class.imagenet_class_index[str(class_idx)][1]
    
    return class_label, class_idx

# Example usage:
image_path = 'assets/Flowers.png'

# Predefined mean and std values for ImageNet models
mean = [0.485, 0.456, 0.406]  # RGB mean for ImageNet
std = [0.229, 0.224, 0.225]   # RGB std for ImageNet

# Predict the class of the image
predicted_label, predicted_class_idx = predict_class(image_path, mean=mean, std=std)

print(f'Predicted class: {predicted_label} (Class index: {predicted_class_idx})')



## Sanity Checking

Now that you can use a trained model for predictions, check to make sure
it makes sense. Even if the testing accuracy is high, it's always good
to check that there aren't obvious bugs. Use `matplotlib` to plot the
probabilities for the top 5 classes as a bar graph, along with the input
image. It should look like this:

<img src='assets/inference_example.png' width=300px>

You can convert from the class integer encoding to actual flower names
with the `cat_to_name.json` file (should have been loaded earlier in the
notebook). To show a PyTorch tensor as an image, use the `imshow`
function defined above.

In [ ]:
# TODO: Display an image along with the top 5 classes
import torch
from torchvision import models, transforms
from PIL import Image
import matplotlib.pyplot as plt
import json
import requests

# Function to process an image (resize, tensor conversion, normalization)
# Function to process an image (resize, tensor conversion, normalization)
def process_image(image_path, input_size=(224, 224), mean=None, std=None):
    image = Image.open(image_path)
    
    # Convert image to RGB if it's in RGBA format (4 channels)
    if image.mode != 'RGB':
        image = image.convert('RGB')
    
    transform_list = [
        transforms.Resize(input_size),
        transforms.ToTensor(),
    ]
    
    if mean is not None and std is not None:
        transform_list.append(transforms.Normalize(mean=mean, std=std))
    
    transform = transforms.Compose(transform_list)
    image_tensor = transform(image)
    
    return image_tensor


# Function to load and predict top 5 classes of an image
def predict_top5(image_path, model=None, input_size=(224, 224), mean=None, std=None):
    # Step 1: Preprocess the image
    processed_image = process_image(image_path, input_size, mean, std)
    
    # Add batch dimension (models expect a batch of images)
    processed_image = processed_image.unsqueeze(0)  # Shape [1, 3, 224, 224]
    
    # Step 2: Load the model (if not passed)
    if model is None:
        model = models.resnet18(pretrained=True)
        model.eval()  # Set to evaluation mode
    
    # Step 3: Predict the top 5 classes
    with torch.no_grad():  # No gradients needed for inference
        output = model(processed_image)
    
    # Step 4: Get the top 5 predicted classes (the indices of the top logits)
    probs, indices = torch.topk(output, 5)  # Get top 5 classes
    
    # Convert probabilities and indices to list
    probs = torch.nn.functional.softmax(probs, dim=1).squeeze().tolist()  # Convert logits to probabilities
    indices = indices.squeeze().tolist()
    
    # Get the class labels (if using ImageNet labels)
    if not hasattr(predict_top5, 'imagenet_class_index'):
        url = 'https://storage.googleapis.com/download.tensorflow.org/data/imagenet_class_index.json'
        response = requests.get(url)
        predict_top5.imagenet_class_index = response.json()
    
    top5_labels = [predict_top5.imagenet_class_index[str(idx)][1] for idx in indices]
    
    return top5_labels, probs, processed_image

# Function to display the image and top 5 predicted classes

def display_image_with_predictions(image_path, top5_labels, probs, image_tensor):
    # Remove batch dimension (i.e., convert shape from [1, 3, 224, 224] to [3, 224, 224])
    image_tensor = image_tensor.squeeze(0)  # Shape will be [3, 224, 224]
    
    # Convert tensor back to PIL image for display
    image = transforms.ToPILImage()(image_tensor)
    
    # Display image
    plt.figure(figsize=(8, 6))
    plt.imshow(image)
    plt.axis('off')  # Turn off axis labels
    
    # Display top 5 predictions
    plt.title("Top 5 Predictions", fontsize=14)
    for i in range(5):
        plt.text(0, i*20, f'{top5_labels[i]}: {probs[i]*100:.2f}%', color='white', fontsize=12)
    
    plt.show()



# Example usage:
image_path = 'assets/Flowers.png'

# Predefined mean and std values for ImageNet models
mean = [0.485, 0.456, 0.406]  # RGB mean for ImageNet
std = [0.229, 0.224, 0.225]   # RGB std for ImageNet

# Get the top 5 predictions for the image
top5_labels, probs, image_tensor = predict_top5(image_path, mean=mean, std=std)

# Display the image along with the top 5 predictions
display_image_with_predictions(image_path, top5_labels, probs, image_tensor)


## Reminder for Workspace users

If your network becomes very large when saved as a checkpoint, there
might be issues with saving backups in your workspace. You should reduce
the size of your hidden layers and train again.

We strongly encourage you to delete these large interim files and
directories before navigating to another page or closing the browser
tab.

In [2]:
# TODO remove .pth files or move it to a temporary `~/opt` directory in this Workspace
